# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

**Dataset imported with:** '$ mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json'

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
databases = mongo.list_database_names()
print(databases)

['admin', 'config', 'local', 'met', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
collections = db.list_collection_names()
print(collections)

['establishments']


In [6]:
# assign the collection to a variable
establishments = db['establishments']

In [7]:
# review a document in the establishments collection
document = establishments.find_one()
pprint(document)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6775d3fd8e548b0163fc842e'),
 'geocode': {'latitude': None, 'longitude': None},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'longitude': None,
 'meta': {'dataSo

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [27]:
# Create a dictionary for the new restaurant data
penang_flavors_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [28]:
# Insert the new restaurant into the collection
result = establishments.insert_one(penang_flavors_data)

In [29]:
# Check that the new restaurant was inserted
if result:
    print("New restaurant inserted:")
    print(result)
else:
    print("Restaurant not found.")

New restaurant inserted:
InsertOneResult(ObjectId('67761a15a1b56b927b1c2782'), acknowledged=True)


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1}
result = establishments.find_one(query, projection)

# Check if the result was found and print the relevant fields
if result:
    print("BusinessTypeID and BusinessType:")
    print(result)
else:
    print("Business type not found.")

BusinessTypeID and BusinessType:
{'_id': ObjectId('6775d3fd8e548b0163fc842f'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
filter_query = {"BusinessName": "Penang Flavours"} 

# Assuming you have found the BusinessTypeID for "Restaurant/Cafe/Canteen"
business_type_id = "1" 

# Specify the update operation
update_operation = {"$set": {"BusinessTypeID": business_type_id}}

# Update the restaurant
result = establishments.update_one(filter_query, update_operation)

In [22]:
# Confirm that the new restaurant was updated
if result.modified_count > 0:
    print(f"Penang Flavors updated successfully with BusinessTypeID {business_type_id}")
else:
    print("No changes made or restaurant not found.")

Penang Flavors updated successfully with BusinessTypeID 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [41]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count = establishments.count_documents(query)
print(f"Number of documents with LocalAuthorityName 'Dover': {count}")

Number of documents with LocalAuthorityName 'Dover': 0


In [42]:
# Delete all documents where LocalAuthorityName is "Dover"
dover_query_delete = {"LocalAuthorityName": "Dover"}
result = establishments.delete_many(dover_query_delete)

In [43]:
# Check if any remaining documents include Dover
print(f"Number of remaining documents with LocalAuthorityName 'Dover': {result.deleted_count}")

Number of remaining documents with LocalAuthorityName 'Dover': 0


In [44]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
if remaining_document:
    print("A document remains in the collection:")
    print(remaining_document)
else:
    print("No documents found in the collection.")

A document remains in the collection:
{'_id': ObjectId('6775d3fd8e548b0163fc842e'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': None, 'latitude': None}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCou

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [45]:
# Change the data type from String to Decimal for longitude and latitude
result = establishments.update_many(
    {},
    [
        {
            '$set': {
                'geocode.latitude': {'$toDecimal': '$geocode.latitude'},
                'geocode.longitude': {'$toDecimal': '$geocode.longitude'}
            }
        }
    ]
)


Use `update_many` to convert `RatingValue` to integer numbers.

In [46]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [47]:
# Change the data type from String to Integer for RatingValue
result = establishments.update_many(
    {},
    [
        {
            '$set': {
                'RatingValue': {'$toInt': '$RatingValue'}
            }
        }
    ]
)

In [48]:
# Check that the coordinates and rating value are now numbers
document = establishments.find_one()

# Check if a document was found and print the relevant fields and their types
if document:
    print("Document details:")
    latitude = document.get('geocode.latitude')
    longitude = document.get('geocode.longitude')
    print(f"Latitude: {latitude} (Type: {type(latitude).__name__})")
    print(f"Longitude: {longitude} (Type: {type(longitude).__name__})")
    print(f"RatingValue: {document['RatingValue']} (Type: {type(document['RatingValue']).__name__})")
else:
    print("No documents found in the collection.")

Document details:
Latitude: None (Type: NoneType)
Longitude: None (Type: NoneType)
RatingValue: 4 (Type: int)
